# Principal Component Analysis

## Preliminaries

First we load in the transcript data from the RSEM output. In this case we select the expected count values, which have not had any library size or gene length normalisation applied.

In [1]:
using Pkg
Pkg.activate("../")
#Pkg.instantiate()
include("../src/ReadExpressionData.jl")
include("../src/Normalisation.jl")
using Gadfly
using Cairo
using Compose
#using Fontconfig
#display("image/png", plot)

 Activating environment at `~/app/Project.toml`
┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /home/osbornejr/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228


In [2]:
## activate equation numbering in markdown latex
macro javascript_str(s) display("text/javascript", s); end
javascript"""
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});
MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],
  ["Reprocess", MathJax.Hub]
);
"""

In [3]:
#read in each set
#raw_counts=RSEM.read_count_data("../data/mayank-de-novo/isoforms",method="expected_count");
raw_counts=RSEM.read_count_data("../data/mayank-per-transcript/isoforms",method="expected_count");;
#create data matrices for each set
data=Array(raw_counts[!,2:25]);



Now we adjust for library size using the upper quartile method (via the R package edgeR):

In [7]:
data=library_size_normalisation(data,"upperquartile");

## The theory

We can use a principal component analysis (PCA) to observe the data and remove possible batch effects. The goal of a PCA is to identify the principal direction and magnitude of variation in a dataset. This information can be used for dimensionality reduction and to improve the quality of the signal ([see here](https://arxiv.org/abs/1404.1100)). 

A dataset with $n$ samples each containing $m$ observations is written as an $m\times n$ matrix $X$. The covariance matrix of $X$ is defined as  

$$C_X=\frac{1}{n-1}XX^T$$

and this matrix that we are interested in optimising (it is also possible to apply the process to the correlation matrix, but we focus on the covariance case for now). Tthis means identifying the signal, represented by the variance of measurements in $X$ (i.e. the diagonal of $C_X$) whilst minimising the redundancy, which means removing any covariance in $X$ (the off-diagonal elements of $C_X$). As $C_X$ is an $m\times m$ symmetric matrix, it is diagonalisable into the form

\begin{align} 
C_X= VDV^T
\label{eq:diag}
\end{align}

where $V$ contains the eigenvectors of $C_X$. If we reorientate $X$ so that $C_X$ is a diagonal matrix, then we will have removed any covariance (redundancy) and identified the largest source of variation (signal) in the dataset.

In cases where $m$ is large, (as is the case with gene expression data) it will be difficult to diagonalise $C_X$ directly, but we can use the concept of a *singular value decomposition* (SVD) to get around this. Any $m \times n$ matrix $X$ can be decomposed into the form    

$$X=A\Sigma B$$

where $A$ is an $m\times m$ orthogonal matrix, $\Sigma$ is an $m \times n$ diagonal matrix, and $B$ is an $n\times n$ orthogonal matrix. If we apply this process to the matrix $Y=\frac{1}{\sqrt{n-1}}X^T$ then we have

$$Y=B\Sigma A$$

and hence

$$Y^TY=(A^T\Sigma B^T)(B\Sigma A)=A^T\Sigma^2A$$

where we have used the fact that $B$ is orthogonal and so $B^T=B^{-1}$. We also have

$$Y^TY=\frac{1}{n-1}XX^T=C_X$$

and so letting $A^T=V$ and $\Sigma^2=D$ we recover equation \eqref{eq:diag}. [EXPORTER'S NOTE: errrghh this eqref works in the notebook but not when exporting!! the equation in question is $C_X= VDV^T$ above].


## Where the confusion begins...
According to [this post](https://stats.stackexchange.com/a/134283/272367), the eigenvectors in $V$ are known as the *principal axes of direction*, and to get the principal components themselves we need to project the data onto these axes using $V'X$. This contradicts what the Shlen tutorial states, which is that the columns of $V$ itself give the principal components.

The difference here is that $V$ is an $m\times m$ matrix (i.e. the same size as $C_X$, massive in our case when $m$ is the number of genes) whilst $V'X$ gives an $m\times n$ matrix i.e. a reproduction of the original data, a truer "transformation" of the data rather than just an insight into it. However, this is further complicated by the fact that most SVD algorithms only return $A$ in a "thin" $m\times k$ form where $k=\min(m,n)$. Correspondingly, projecting onto $V=A^T$ does not give a fully transformed dataset but rather a $k\times n$ matrix, which will be reduced from the original when $n<m$. In the case of gene expression data, it is almosts always the case that $m>>>n$, and so the resultant transformed matrix has reduced the set of normalised genes down to the number of samples. Clearly this is not desirable and highlights that perhaps this form of PCA is not useful in the case of gene expression counts-- the concept of removing any covariance/correlation between genes is counter to what we are trying to achieve anyway. Perhaps some assumptions that PCA relies upon (i.e. linearity or that redunancy should be removed) are not valid here. 

The main source of trouble seems to be that most PCAs are framed around datasets where the number of samples greatly outweighs the number of measurements (the opposite of our situation). Perhaps it is more useful for us to analyse across samples, and seek dimensionality reduction there-- but this does seem to contradict most theorectical descriptions of PCA. There is some genomic-specific material ([see here](https://benthamopen.com/ABSTRACT/TOBIOIJ-7-19)) that indicates that PCA can be done per-sample *or* per-gene, which seems to confuse the issue more.


In [5]:
PCs,D,per_sample=pca(data)
p=pca_plot(per_sample,3);
draw(SVG("test.svg"),p)
#compose(p)l
p=pca_plot(PCs,1);
draw(SVG("test_PC.svg"),p)
#compose(p);
PCs,D_1,per_sample=pca(data')
p=pca_plot(PCs,3);
draw(SVG("test_per_sample.svg"),p);

false

## Examples
In short, the options for PCA on gene expression data seem to be:

a) interpret the columns of $V$ as the principal components themselves. With the "thin" version of SVD, this gives us $n$ components each with $m$ dimensions. Naively, this gives us a matrix that is the same dimensions as the original data, but it doesn't seem a correct to interpret this as a data transformation seeing as the we are really just looking at a (truncated) set of the eigenvectors of the covariance matrix.


b) interpret $V$ as the principal axes of direction, and take the columns of $V'X$ as the principal components. In this case the "thin" version applied to $X$ would give us an $n\times n$ matrix, which seemingly would be intrepreted in our case as the gene set reduced down to $n$ dimensions over $n$ principal components. There is also [some reference](https://blog.bioturing.com/2018/06/14/principal-component-analysis-explained-simply/) to this interpretation being a collation of per-gene data to give per-sample scores, but I am unsure if this is correct.

![plot_1](test.svg "Projecting onto the principal axes")
[EXPORTER'S NOTE: and again the plots aren't being exported. I'll send them separately labelled "plot_1" and "plot_2".]

c) Using a full SVD to generate $V$ (which would mean $V'X$ is $m\times n$, as in the original data) might be the most desirable way to actually transform the data. But as the full SVD chooses arbitrary eigenvectors to fill the $m-k$ dimensions, it is not clear how this would give meaningful results beyond the first $n$ dimensions anyway. In any case, this process is too computationally demanding with such large $m$ (Julia cannot store the $m\times m$ eigenvector matrix in memory).
 
d) Transpose the input data, and apply a PCA on the samples, rather than the genes. The eigenvalue matrix is then $n \times n$ and can be used directly as in a) to give principal components. However, it would seem that PCA is more of a QC step checking for outlying samples rather than having any data transformation or dimension reduction usage. It would seem counterintuitive to further reduce our already low number of samples! 

![plot_2](test_per_sample.svg "Per-sample PCA" )


e) As in d), but transform the data using the eigenvector matrix. This gives a $m\times n$ matrix that might actually make sense as a proper data transformation based on per-sample principal components. Does this sound like an effective usage of PCA? To be honest it was a strategy that hadn't crossed my mind until having written all of the other options up, but maybe it makes the most sense in the context of gene expression data? It is not something I have seen done before though.



*Because the most obvious effect we can look for is the difference between the polyA+ data and the polyA- data, components are coloured based on this (where possible).* 


## Conclusion
It is hard to know what to conclude here, other than asking whether in the end PCA is more trouble than it is worth? Its use is widespread in bioinformatics, but often just to generate biplots or look at the data rather than actually transforming it. It is clear now to me that PCA is a much more powerful tool than that, but fear that if misused it would do more harm than good to the analysis.